In [2]:
# seeds 
seed_import = 265
seed_QSGD = 155
seed_customFreq_2_5 = 5454
seed_Custom_linear_16 = 38763
seed_Custom_linear = 5425
seed_FP8 = 29855
seed_FL_implementation = 546984

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime
import random


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int
np.random.seed(seed_import)
from scipy.fftpack import dct, idct


In [3]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Selected Schemes

In [4]:
class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")
Main.include("src/compressors/fp8.jl")

number_of_selected_schemes = 4
column_names = ["accuracy","time","efficiency"]
metrics_cell = np.zeros((number_of_selected_schemes, len(column_names)), dtype=object)

# list = []
# list.append(dict)
# dict[qs]={{a,t,e}





#______________QSGD________________
class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        np.random.seed(seed_QSGD)
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle
        

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=seed_QSGD, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)

# cmp = GZip(1)

#______________FP8________________
class FP8(Compressor):
    def __init__(self):
        np.random.seed(seed_FP8)
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=seed_FP8, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
# cmp = FP8()




#______________Custom Linear________________
class Custom_linear():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(seed_Custom_linear)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        
    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec

# data_type = np.uint8
# cmp = Custom_linear(data_type)


class Custom_linear_16():
    def __init__(self, quantization_type = np.uint16):
        np.random.seed(seed_Custom_linear_16)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec






#______________Frequency Domain Quantization________________



class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(seed_customFreq_2_5)
        self.precision_levels = precision_levels
        self.linearQuant = Custom_linear()
        #quantization_level from 0 to 100  
              
    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
# cmp = customFreq_2_5()





e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.10.4+0.x64.w64.mingw32. You currently have `1.10.2+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.4+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


In [5]:
from sklearn.preprocessing import MinMaxScaler

def get_time_efficiency_acc(cmp):

    df_acc_loss = pd.read_csv("E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_ACC_LOSS_sheet.csv")

    acc_loss = df_acc_loss[df_acc_loss['Epoch']==9]
    acc_loss = acc_loss.drop(['Epoch'], axis=1)
    df_time_eff = pd.read_csv("E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_TIME_EFF_sheet.csv")
    df_properties = pd.merge(acc_loss, df_time_eff, on='scheme_name')
    # print(df_properties)
    scaler = MinMaxScaler()
    df_properties_scaled = df_properties.copy()
    df_properties_scaled[['Acc', 'Time', 'Efficiency']] = scaler.fit_transform(df_properties_scaled[['Acc', 'Time', 'Efficiency']])
    weights = {
        'Acc': 0.5,
        'Time': 0.3,
        'Efficiency': 0.2
    }

    # Compute the combined measure
    df_properties_scaled['Combined_Score'] = (
        weights['Acc'] * df_properties_scaled['Acc'] +
        weights['Time'] * (1 - df_properties_scaled['Time']) +  
        weights['Efficiency'] * df_properties_scaled['Efficiency']
    )


    name = cmp.__class__.__name__

    time = df_properties[df_properties['scheme_name']==name]['Time'].values[0]
    acc = df_properties[df_properties['scheme_name']==name]['Acc'].values[0]
    eff = df_properties[df_properties['scheme_name']==name]['Efficiency'].values[0]

    combined = df_properties_scaled[df_properties_scaled['scheme_name']==name]['Combined_Score'].values[0]


    return name, acc, time, eff, combined
    
    


# Federated Implementation

In [6]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp_list, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		
		np.random.seed(seed_FL_implementation)
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp = None
		self.cmp_list = cmp_list
		self.client_ids = list(range(0,number_of_clients))

		self.bw_low = 20		
		self.bw_high = 2500

		self.traffic_low = 1
		self.traffic_high = 51
		# self.bw_traffic = [[random.randint(self.bw_low, self.bw_high), random.randint(1,50)] for _ in range(number_of_clients)]
	
		bandwidths = np.random.randint(self.bw_low, self.bw_high + 1, size=self.number_of_clients)
		traffic = np.random.randint(self.traffic_low, self.traffic_high, size=self.number_of_clients)
		self.bw_traffic = list(zip(bandwidths, traffic))


#a list of tuple of shape(list) number_of_clients

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        

        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100

		weight_array = np.array(weights)
		try:
			assert(weight_array.shape == (6,2))
		except:
			print('Scheme',self.cmp)
			print('weight_array.shape',weight_array.shape)

		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights




	def get_client_bw_and_traffic(self,client_id):
		bw,traffic = self.bw_traffic[client_id]
		return bw,traffic
	


	def quanization_scheme(self,bw,traffic):
		normalized_bw = bw/(self.bw_high-self.bw_low)
		normalized_traffic = traffic/(self.traffic_high-self.traffic_low) 

		client_condition = normalized_bw * 50 + normalized_traffic * 50

		data = np.arange(0,100)
		percentile_33 = np.percentile(data, 33)
		percentile_66 = np.percentile(data, 66)

		if client_condition<=percentile_33:
			selected_cmp = self.cmp_list[0]
		elif percentile_33 < client_condition <= percentile_66:
			selected_cmp = self.cmp_list[1]
		else:
			selected_cmp = self.cmp_list[2]			
		return selected_cmp
	
		

	def client_generator(self, train_x, train_y):
		
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()		
		return weights, running_loss, start_size,end_size
	
	
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for selection_index,(dataset_x, dataset_y )in enumerate(clients):
				client_id = selections[selection_index]
				bw,traffic = self.get_client_bw_and_traffic(client_id)
				self.cmp = self.quanization_scheme(bw,traffic)
				# print(self.cmp)
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			self.test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(self.test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight
	
	def get_acc(self):
		return self.test_acc

	


#%%

In [7]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
# epoch_time = time.time()
tic = datetime.now()
filename = "saved_models_x"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)

cmp_list = [customFreq_2(),QSGD(2,True),Custom_linear_16()]
# cmp_list = [customFreq_2(),customFreq_2(),Custom_linear_16()]
d = {}

for cmp in cmp_list:
    name, acc, time, eff, combined = get_time_efficiency_acc(cmp)
    d[cmp] = eff
    
print(d)
sorted_schemes = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
print(sorted_schemes.keys())
# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp_list=list(sorted_schemes.keys()), filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", datetime.now()-tic)

{<__main__.customFreq_2 object at 0x000002A89469A2C0>: 0.2758620689655172, <__main__.QSGD object at 0x000002A89469BCA0>: 0.5288461538461539, <__main__.Custom_linear_16 object at 0x000002A894698B50>: 0.4137931034482758}
dict_keys([<__main__.QSGD object at 0x000002A89469BCA0>, <__main__.Custom_linear_16 object at 0x000002A894698B50>, <__main__.customFreq_2 object at 0x000002A89469A2C0>])


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_6184\577508066.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
C:\Users\Asus\AppData\Local\Temp\ipykernel_6184\577508066.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30154}:   2%|▏         | 4/164 [00:05<02:28,  1.08it/s]

Scheme <__main__.customFreq_2 object at 0x000002A89469A2C0>
weight_array.shape (0,)


{'Epoch': 1, 'Loss': 2.30149}:   7%|▋         | 11/164 [00:05<00:29,  5.19it/s]

Scheme <__main__.customFreq_2 object at 0x000002A89469A2C0>
weight_array.shape (0,)


...

Scheme <__main__.customFreq_2 object at 0x000002A89469A2C0>
weight_array.shape (0,)


{'Epoch': 1, 'Loss': 2.30101}:  96%|█████████▌| 157/164 [00:21<00:00,  8.45it/s]

Scheme <__main__.customFreq_2 object at 0x000002A89469A2C0>
weight_array.shape (0,)


{'Epoch': 1, 'Loss': 2.30101}: 100%|██████████| 164/164 [00:22<00:00,  7.39it/s]
C:\Users\Asus\AppData\Local\Temp\ipykernel_6184\577508066.py:109: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_weights = np.array(all_weights)


ValueError: operands could not be broadcast together with shapes (6,2) (0,) 

In [ ]:
{<__main__.customFreq_2 object at 0x0000024FB139E800>: 0.2758620689655172, <__main__.QSGD object at 0x0000024FC38E37C0>: 0.5288461538461539, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>: 0.4137931034482758}
dict_keys([<__main__.QSGD object at 0x0000024FC38E37C0>, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>, <__main__.customFreq_2 object at 0x0000024FB139E800>])
  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:22<00:00,  7.26it/s]
Test Accuracy: 0.10228
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 1.95148}: 100%|██████████| 164/164 [00:24<00:00,  6.78it/s]
Test Accuracy: 0.47029
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.74089}: 100%|██████████| 164/164 [00:26<00:00,  6.19it/s]
Test Accuracy: 0.73742
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.68466}: 100%|██████████| 164/164 [00:24<00:00,  6.76it/s]
Test Accuracy: 0.75613
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.61751}: 100%|██████████| 164/164 [00:22<00:00,  7.16it/s]
Test Accuracy: 0.83759
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.60218}: 100%|██████████| 164/164 [00:24<00:00,  6.57it/s]
Test Accuracy: 0.853
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.59334}: 100%|██████████| 164/164 [00:24<00:00,  6.79it/s]
Test Accuracy: 0.8574
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.58873}: 100%|██████████| 164/164 [00:27<00:00,  6.05it/s]
Test Accuracy: 0.86199
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53745}: 100%|██████████| 164/164 [00:23<00:00,  7.00it/s]
Test Accuracy: 0.86518
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.50526}: 100%|██████████| 164/164 [00:27<00:00,  5.86it/s]
Test Accuracy: 0.94933
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  0:04:27.716845

In [37]:
print(agg_weights)

print(m_8.get_acc())

None
tensor(0.9491)


# Q learning

In [1]:
# # seeds 
# seed_import = 265
# seed_QSGD = 155
# seed_customFreq_2_5 = 5454
# seed_Custom_linear_16 = 38763
# seed_Custom_linear = 5425
# seed_FP8 = 29855
# seed_FL_implementation = 546984

yes